## Import stuff

In [6]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


from tensorflow.python.client import device_lib
device_lib.list_local_devices() 


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13118909115092292733]

## Load training data from csv


In [7]:
csv = pd.read_csv("train.csv")

Y = csv["y"].tolist()
# load columns x1 to x13 as an array of vectors
X = np.array(csv[["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13"]].values)

X = np.array(X,  dtype=float)
Y = np.array(Y,  dtype=float)

X = tf.cast(X, tf.float32)
Y = tf.cast(Y, tf.float32)



## Create the model and compile it, with loss and optimizer functions

In [11]:
reg_params = [0.1, 1, 10, 100, 200]
res = np.zeros(5)

for i in range(5):

    reg = reg_params[i]
    all_RMSE = np.zeros(10)

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=10, shuffle=True)

    # K-fold Cross Validation model evaluation
    fold_no = 0
    for train, test in kfold.split(X, Y):

        # create a model with 10-fold cross validation and regularization parameter of 0.1
        
        model = tf.keras.Sequential([
            layers.Dense(1, activation='relu', input_shape=[13],kernel_regularizer=tf.keras.regularizers.L1(reg),activity_regularizer=tf.keras.regularizers.L2(reg)),
        
        ])

        model.compile(optimizer='adam', loss='mse', metrics=['mse'])

        history = model.fit(X, Y, epochs=10) # , verbose=False
        print("Finished training the model")

        pred = model.predict(X)

        RMSE = mean_squared_error(Y, pred)**0.5

        
        all_RMSE[fold_no] = RMSE

        fold_no+=1

    res[i] = np.mean(all_RMSE)
    # print("RMSE average with param ", reg,": ", np.average(all_RMSE))

print(res)


Epoch 1/10
5/5 [==============================] - 0s 400us/step - loss: 567.8966 - mse: 567.5457
Epoch 2/10
5/5 [==============================] - 0s 400us/step - loss: 567.8901 - mse: 567.5457
Epoch 3/10
5/5 [==============================] - 0s 400us/step - loss: 567.8837 - mse: 567.5456
Epoch 4/10
5/5 [==============================] - 0s 400us/step - loss: 567.8771 - mse: 567.5457
Epoch 5/10
5/5 [==============================] - 0s 601us/step - loss: 567.8706 - mse: 567.5457
Epoch 6/10
5/5 [==============================] - 0s 600us/step - loss: 567.8641 - mse: 567.5456
Epoch 7/10
5/5 [==============================] - 0s 601us/step - loss: 567.8577 - mse: 567.5456
Epoch 8/10
5/5 [==============================] - 0s 401us/step - loss: 567.8511 - mse: 567.5456
Epoch 9/10
5/5 [==============================] - 0s 601us/step - loss: 567.8446 - mse: 567.5456
Epoch 10/10
5/5 [==============================] - 0s 600us/step - loss: 567.8381 - mse: 567.5457
Finished training the model
E

# Export results

In [13]:
# write the RMSE to file "submit.csv"
with open("submit.csv", "w") as f:
    for i in res:
        f.write(str(i) + "\n")

